## Step 1: Get the background and distraction/cofounding elements

In [ ]:
%%bash

#------------------------------------------------------------------------------
# wget the background images from http://graphics.cs.cmu.edu/projects/whatMakesParis/
#------------------------------------------------------------------------------

# Directory to save the .jpg files
directory="backgrounds"

# Create the directory if it doesn't exist
mkdir -p "$directory"

# Loop through numbers 1 to 100
for i in {1..100}
do
  # Construct the URL for the current file
  url="http://graphics.cs.cmu.edu/projects/whatMakesParis/testimgs/${i}.jpg"

  # Download the file to the specified directory
  wget "$url" -P "$directory"

  echo "Downloaded ${i}.jpg"
done

In [ ]:
%%bash

#------------------------------------------------------------------------------
# wget distractions
#------------------------------------------------------------------------------

# Create the directory if it doesn't exist
mkdir -p "distractions"

# download the lamps
for i in $(seq 14075 14094); do
  wget "http://graphics.cs.cmu.edu/projects/whatMakesParis/result/alldiscpatchimg[]/$i.jpg" -P distractions
done

# download the wall
for i in $(seq 13915 13934); do
  wget "http://graphics.cs.cmu.edu/projects/whatMakesParis/result/alldiscpatchimg[]/$i.jpg" -P distractions
done

## Step 2: Download and Process the drone images

In [ ]:
%%bash

#------------------------------------------------------------------------------
# wget images from https://freepngimg.com/electronics/drone into drones directory
#------------------------------------------------------------------------------

# Create the directory if it doesn't exist
mkdir -p "drones"

# download images
wget https://freepngimg.com/download/drone/21576-6-drone-image.png -P drones
wget https://freepngimg.com/download/drone/21682-6-drone-picture.png -P drones

In [ ]:
#------------------------------------------------------------------------------
# There are some images in drones directory. However, some of the images have transparent pixels around them, 
# instead of the border of iamge is fit to the object itself. Please remove these pixels. Then save as png file
#------------------------------------------------------------------------------

import cv2
import os
import random
import numpy as np
from PIL import Image

def remove_transparent_pixels(image_path, output_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if img is None:
        print(f"Error: Could not open or read image: {image_path}")
        return

    if img.shape[2] == 4:  # Check if the image has an alpha channel
        # Find the bounding box of non-transparent pixels
        alpha = img[:, :, 3]
        rows = np.any(alpha, axis=1)
        cols = np.any(alpha, axis=0)
        rmin, rmax = np.where(rows)[0][[0, -1]]
        cmin, cmax = np.where(cols)[0][[0, -1]]

        # Crop the image to the bounding box
        img = img[rmin:rmax+1, cmin:cmax+1, :]

    # Save as PNG
    cv2.imwrite(output_path, img)


# Iterate through the images in the 'drones' directory
drones_dir = "drones"
output_dir = "drones_processed"

os.makedirs(output_dir, exist_ok=True)

import numpy as np

for filename in os.listdir(drones_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(drones_dir, filename)
        output_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".png")
        remove_transparent_pixels(image_path, output_path)
        print(f"Processed: {filename}, saved as: {output_path}")


## Step 3: Create the synthetic dataset

In [ ]:
#------------------------------------------------------------------------------
# Create the synthetic dataset
#------------------------------------------------------------------------------

# --- Configuration ---
background_dir = 'backgrounds'
drone_dir = 'drones_processed'
distraction_dir = 'distractions'
output_dir = 'synthetic'
output_images_dir = os.path.join(output_dir, 'images')
output_labels_dir = os.path.join(output_dir, 'labels')

# Number of drones to insert per background image copy
N = 3  # Change as needed

# Number of copies to create per background image
M = 20  # Change as needed

# Number of distractions to insert per background image copy
K = 3  # Change as needed

# Target height for the background images
target_bg_height = 960

# Create output directories if they do not exist
os.makedirs(output_images_dir, exist_ok=True)
os.makedirs(output_labels_dir, exist_ok=True)

# Function to get image paths from a directory
def get_image_paths(directory):
    valid_exts = ('.png', '.jpg', '.jpeg', '.bmp')
    return [os.path.join(directory, f) for f in os.listdir(directory)
            if f.lower().endswith(valid_exts)]

background_paths = get_image_paths(background_dir)
drone_paths = get_image_paths(drone_dir)
distraction_paths = get_image_paths(distraction_dir)

# Function to rotate an image (with possible alpha channel)
def rotate_image(image, angle):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M_rot = cv2.getRotationMatrix2D(center, angle, 1.0)
    # Compute new bounding dimensions
    cos = np.abs(M_rot[0, 0])
    sin = np.abs(M_rot[0, 1])
    new_w = int((h * sin) + (w * cos))
    new_h = int((h * cos) + (w * sin))
    # Adjust the rotation matrix to take into account translation
    M_rot[0, 2] += (new_w / 2) - center[0]
    M_rot[1, 2] += (new_h / 2) - center[1]
    rotated = cv2.warpAffine(image, M_rot, (new_w, new_h), flags=cv2.INTER_LINEAR,
                             borderMode=cv2.BORDER_CONSTANT, borderValue=(0, 0, 0, 0))
    return rotated

# Process each background image
for bg_path in background_paths:
    bg_orig = cv2.imread(bg_path)
    if bg_orig is None:
        continue  # Skip if unable to load image

    # Rescale the background image so that its height is target_bg_height while keeping the aspect ratio
    orig_H, orig_W = bg_orig.shape[:2]
    scale_factor = target_bg_height / orig_H
    new_W = int(orig_W * scale_factor)
    bg_orig = cv2.resize(bg_orig, (new_W, target_bg_height), interpolation=cv2.INTER_AREA)

    base_name = os.path.splitext(os.path.basename(bg_path))[0]

    # Create M copies for each background image
    for m in range(M):
        bg = bg_orig.copy()
        H, W = bg.shape[:2]
        annotations = []  # To store YOLOv8 annotations for drones only

        # Insert K distraction images (not annotated)
        for i in range(K):
            distraction_path = random.choice(distraction_paths)
            distraction = cv2.imread(distraction_path, cv2.IMREAD_UNCHANGED)
            if distraction is None:
                continue

            # Set target height between 100 and 200 pixels for distraction
            target_height = random.randint(100, 200)
            scale = target_height / distraction.shape[0]
            new_width = int(distraction.shape[1] * scale)
            new_height = int(distraction.shape[0] * scale)
            distraction = cv2.resize(distraction, (new_width, new_height), interpolation=cv2.INTER_AREA)

            # Apply a random rotation between -3 and 3 degrees.
            angle = random.uniform(-3, 3)
            distraction = rotate_image(distraction, angle)
            d_h, d_w = distraction.shape[:2]

            # If the distraction is larger than the background, scale it down further.
            if d_w >= W or d_h >= H:
                factor = min(W / d_w, H / d_h) * 0.9
                distraction = cv2.resize(distraction, (int(d_w * factor), int(d_h * factor)), interpolation=cv2.INTER_AREA)
                d_h, d_w = distraction.shape[:2]

            # Choose a random location where the distraction fully fits within the background.
            max_x = W - d_w
            max_y = H - d_h
            if max_x <= 0 or max_y <= 0:
                continue  # Skip if the distraction cannot be placed properly
            x_offset = random.randint(0, max_x)
            y_offset = random.randint(0, max_y)

            # Paste the distraction onto the background.
            if distraction.shape[2] == 4:
                overlay_rgb = distraction[:, :, :3]
                alpha_mask = distraction[:, :, 3] / 255.0
                roi = bg[y_offset:y_offset+d_h, x_offset:x_offset+d_w]
                for c in range(3):
                    roi[:, :, c] = (roi[:, :, c] * (1 - alpha_mask) +
                                    overlay_rgb[:, :, c] * alpha_mask)
                bg[y_offset:y_offset+d_h, x_offset:x_offset+d_w] = roi
            else:
                bg[y_offset:y_offset+d_h, x_offset:x_offset+d_w] = distraction

        # Insert N drone images (annotated)
        for i in range(N):
            drone_path = random.choice(drone_paths)
            drone = cv2.imread(drone_path, cv2.IMREAD_UNCHANGED)
            if drone is None:
                continue

            # Set the drone's target height to a random value between 40 and 70 pixels.
            target_height = random.randint(40, 70)
            scale = target_height / drone.shape[0]
            new_width = int(drone.shape[1] * scale)
            new_height = int(drone.shape[0] * scale)
            drone = cv2.resize(drone, (new_width, new_height), interpolation=cv2.INTER_AREA)

            # Apply a random rotation between -3 and 3 degrees.
            angle = random.uniform(-3, 3)
            drone = rotate_image(drone, angle)
            drone_h, drone_w = drone.shape[:2]

            # If the drone image is larger than the background, scale it down further.
            if drone_w >= W or drone_h >= H:
                factor = min(W / drone_w, H / drone_h) * 0.9
                drone = cv2.resize(drone, (int(drone_w * factor), int(drone_h * factor)), interpolation=cv2.INTER_AREA)
                drone_h, drone_w = drone.shape[:2]

            # Choose a random location where the drone fully fits within the background.
            max_x = W - drone_w
            max_y = H - drone_h
            if max_x <= 0 or max_y <= 0:
                continue  # Skip if the drone cannot be placed properly
            x_offset = random.randint(0, max_x)
            y_offset = random.randint(0, max_y)

            # Paste the drone onto the background.
            if drone.shape[2] == 4:
                overlay_rgb = drone[:, :, :3]
                alpha_mask = drone[:, :, 3] / 255.0
                roi = bg[y_offset:y_offset+drone_h, x_offset:x_offset+drone_w]
                for c in range(3):
                    roi[:, :, c] = (roi[:, :, c] * (1 - alpha_mask) +
                                    overlay_rgb[:, :, c] * alpha_mask)
                bg[y_offset:y_offset+drone_h, x_offset:x_offset+drone_w] = roi
            else:
                bg[y_offset:y_offset+drone_h, x_offset:x_offset+drone_w] = drone

            # Compute YOLOv8 annotation (format: class_id x_center y_center width height normalized).
            x_center = (x_offset + drone_w / 2) / W
            y_center = (y_offset + drone_h / 2) / H
            width_norm = drone_w / W
            height_norm = drone_h / H
            annotations.append(f"0 {x_center:.6f} {y_center:.6f} {width_norm:.6f} {height_norm:.6f}")

        # Save the new image and its corresponding YOLO annotation text file.
        output_img_path = os.path.join(output_images_dir, f"{base_name}_{m}.jpg")
        output_txt_path = os.path.join(output_labels_dir, f"{base_name}_{m}.txt")
        cv2.imwrite(output_img_path, bg)
        with open(output_txt_path, 'w') as f:
            for ann in annotations:
                f.write(ann + '\n')

print("Dataset generation completed.")
